In [1]:
import time
import requests
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.proxy import Proxy, ProxyType
import numpy as np
import aiohttp
import asyncio
import socks
import nest_asyncio
import random
import logging
import tenacity
from IPython.display import HTML
from fake_useragent import UserAgent
import itertools
import re

# Парсинг сайтов kns и regard
# Алтухов Николай

In [3]:
# Выбор метода парсинга, aiohttp или selenium
method = 'aiohttp'

In [64]:
# ссылки страниц поиска телефонов с сайта kns
urls2 = [f'https://www.kns.ru/multi/catalog/smartfony/_v-nalichii/page{i}/' for i in range(1,22+1)]


In [87]:
urls = urls2
batch_size = 3

In [89]:
# Замер времени выполнения кода
start_time = time.time()
# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Использование nest_asyncio для запуска асинхронного кода в ноутбуке Jupyter
nest_asyncio.apply()


# Определение асинхронной функции для получения контента страницы с помощью aiohttp
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(aiohttp.ClientError)
)
async def fetch(session, url, ):
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'ru-RU,ru;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1'
    }
    proxy = "http://127.0.0.1:50445"
    async with session.get(url, headers=headers, proxy=proxy) as response:
        response.raise_for_status()  # Это вызовет исключение для ненулевых статусов
        return await response.text()

# Определение асинхронной функции для получения контента страницы с помощью Selenium
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(Exception))

async def fetch_with_selenium(url, proxy="127.0.0.1:50445"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        
    driver_path = 'C:\\Program Files\\Google\\Chrome\\Application\\chromedriver-win64\\chromedriver.exe'
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        await asyncio.sleep(random.uniform(6, 12))  # Рандомная задержка
        html_content = driver.page_source
    finally:
        driver.quit()
    return html_content

# Определение асинхронной функции для парсинга контента страницы с помощью aiohttp
async def parse_with_aiohttp(url):
    async with aiohttp.ClientSession() as session:
        html_content = await fetch(session, url)
        await asyncio.sleep(random.uniform(1, 3))
        
        soup = BeautifulSoup(html_content, 'lxml')
        items = soup.select('a.name.d-block')


        listings = []
        for idx, item in enumerate(items):
            title = item.get_text(strip=True)
            link = item['href']

            # Проверяем, что элементы rating_oblast1, text1 и gorod1 существуют и добавляем их в listings
            #price = prices[idx].get_text(strip=True) if idx < len(price) else None


            listings.append({'Название': title,'Ссылка':link})
        return listings
    
# Определение асинхронной функции для парсинга контента страницы с помощью selenium
async def parse_with_selenium(url):
    html_content = await fetch_with_selenium(url)
    soup = BeautifulSoup(html_content, 'lxml')
    items = soup.select('a.name.d-block')


    listings = []
    for idx, item in enumerate(items):
        title = item.get_text(strip=True)
        link = item['href']

        listings.append({'Название': title,'Ссылка':link})
    return listings

# Определение асинхронной функции для выполнения парсинга в пакетном режиме
async def batch_executor(urls, batch_size=2, parse_func=None):
    results = []
    for i in range(0, len(urls), batch_size):
        batch = urls[i:i + batch_size]
        tasks = [parse_func(url) for url in batch]
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        for result in batch_results:
            if isinstance(result, Exception):
                logging.error(f"An error occurred: {result}")
            else:
                results.extend(result)
    return results

# Определение асинхронной функции для выполнения парсинга всех страниц
async def main(urls, batch_size,parse_func):
    all_listings = await batch_executor(urls, batch_size, parse_func=parse_func)  # Указываем размер партии
    df = pd.DataFrame(all_listings)
    return df





# Выполнение парсинга с использованием aiohttp или Selenium в зависимости от значения параметра method
async def run(urls, method, batch_size):
    try:
        start_time = time.time()

        if method == 'aiohttp':
            df = await main(urls, batch_size, parse_func=parse_with_aiohttp)
        elif method == 'selenium':
            logging.getLogger('selenium.webdriver').setLevel(logging.WARNING)
            df = await main(urls, batch_size, parse_func=parse_with_selenium)

        elapsed_time = time.time() - start_time
        print(f"Execution time: {elapsed_time:.2f} seconds")

        return df
    except Exception as e:
        logger.error(f"Error in run function: {e}")
        raise

if __name__ == "__main__":
    df = asyncio.run(run(urls, method, batch_size))
    

    
print(f"Время выполнения функции: {time.time() - start_time} секунд")

Execution time: 42.45 seconds
Время выполнения функции: 42.450247287750244 секунд


In [121]:
df

,Название,Ссылка,kns,kns_product
0,Смартфон Xiaomi Redmi Note 13 Pro+ 5G 12/512GB...,/product/smartfon-xiaomi-redmi-note-13-proplus...,https://www.kns.ru,https://www.kns.ru/product/smartfon-xiaomi-red...
1,Смартфон Apple iPhone 15 128GB Blue MTLG3CH/A,/product/smartfon-apple-iphone-15-128gb-blue-m...,https://www.kns.ru,https://www.kns.ru/product/smartfon-apple-ipho...
2,Смартфон Apple iPhone 15 128GB Yellow MTP23HN/A,/product/smartfon-apple-iphone-15-128gb-yellow...,https://www.kns.ru,https://www.kns.ru/product/smartfon-apple-ipho...
3,Смартфон Samsung Galaxy XCover 7 6/128GB Black...,/product/smartfon-samsung-galaxy-xcover-7-6-12...,https://www.kns.ru,https://www.kns.ru/product/smartfon-samsung-ga...
4,Смартфон Samsung Galaxy A34 6/128GB Green SM-A...,/product/smartfon-samsung-galaxy-a34-128gb-gre...,https://www.kns.ru,https://www.kns.ru/product/smartfon-samsung-ga...
...,...,...,...,...
654,Смартфон Samsung Galaxy S24 8/256GB Yellow SM-...,/product/smartfon-samsung-galaxy-s24-8-256gb-y...,https://www.kns.ru,https://www.kns.ru/product/smartfon-samsung-ga...
655,Смартфон Samsung Galaxy S24 8/256GB Violet SM-...,/product/smartfon-samsung-galaxy-s24-8-256gb-v...,https://www.kns.ru,https://www.kns.ru/product/smartfon-samsung-ga...
656,Смартфон Vivo V30 Lite 8/256GB Green,/product/smartfon-vivo-v30-lite-8-256gb-green/,https://www.kns.ru,https://www.kns.ru/product/smartfon-vivo-v30-l...
657,Смартфон Vivo V30 Lite 8/256GB Black,/product/smartfon-vivo-v30-lite-8-256gb-black/,https://www.kns.ru,https://www.kns.ru/product/smartfon-vivo-v30-l...


In [101]:
df['kns'] = 'https://www.kns.ru'
df['kns_product'] = df['kns'] + df['Ссылка']

In [105]:
df_links = df.copy()

In [127]:
df_links.to_excel('kns ссылки на телефоны.xlsx', index=False)

In [133]:
# определение ссылок на все телефоны сайта kns
urls = df_links['kns_product'].to_list()
batch_size = 3

In [141]:
# Замер времени выполнения кода
start_time = time.time()
# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Использование nest_asyncio для запуска асинхронного кода в ноутбуке Jupyter
nest_asyncio.apply()


# Определение асинхронной функции для получения контента страницы с помощью aiohttp
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(aiohttp.ClientError)
)
async def fetch(session, url, ):
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'ru-RU,ru;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1'
    }
    proxy = "http://127.0.0.1:50445"
    async with session.get(url, headers=headers, proxy=proxy) as response:
        response.raise_for_status()  # Это вызовет исключение для ненулевых статусов
        return await response.text()

# Определение асинхронной функции для получения контента страницы с помощью Selenium
@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(Exception))

async def fetch_with_selenium(url, proxy="127.0.0.1:50445"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        
    driver_path = 'C:\\Program Files\\Google\\Chrome\\Application\\chromedriver-win64\\chromedriver.exe'
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        await asyncio.sleep(random.uniform(6, 12))  # Рандомная задержка
        html_content = driver.page_source
    finally:
        driver.quit()
    return html_content

# Определение асинхронной функции для парсинга контента страницы с помощью aiohttp
async def parse_with_aiohttp(url):
    async with aiohttp.ClientSession() as session:
        html_content = await fetch(session, url)
        await asyncio.sleep(random.uniform(1, 3))
        
        soup = BeautifulSoup(html_content, 'lxml')
        listings = []
        def get_text_or_none(selector):
            element = soup.select_one(selector)
            return element.get_text(strip=True) if element else None
        price = get_text_or_none('span.price-val')
        PN = get_text_or_none('div[data-id="2066404833"]')
        main_name = get_text_or_none('h1[itemprop="name"]')
        model_name = get_text_or_none('div[data-id="2066404832"]')
        RAM = get_text_or_none('div[data-id="2066402682"] span.text-nounderline.f-link-block.text-primary.cursor-pointer.word-break-all')
        ROM = get_text_or_none('div[data-id="2066402683"] span.text-nounderline.f-link-block.text-primary.cursor-pointer.word-break-all')
        description = get_text_or_none('div.goods-annt.mt-2 span.description')

        listings.append({'Название': main_name,'Цена':price, 'Код производителя':PN, 'Модель':model_name,"RAM":RAM, 'ROM':ROM, 'Описание':description})
        return listings
    
# Определение асинхронной функции для парсинга контента страницы с помощью Selenium
async def parse_with_selenium(url):
    html_content = await fetch_with_selenium(url)
    soup = BeautifulSoup(html_content, 'lxml')
    items = soup.select('a.name.d-block')


    listings = []
    for idx, item in enumerate(items):
        title = item.get_text(strip=True)
        link = item['href']

        # Проверяем, что элементы rating_oblast1, text1 и gorod1 существуют и добавляем их в listings
        #price = prices[idx].get_text(strip=True) if idx < len(price) else None


        listings.append({'Название': title,'Ссылка':link})
    return listings

# Определение асинхронной функции для выполнения парсинга в пакетном режиме
async def batch_executor(urls, batch_size=2, parse_func=None):
    results = []
    for i in range(0, len(urls), batch_size):
        batch = urls[i:i + batch_size]
        tasks = [parse_func(url) for url in batch]
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        for result in batch_results:
            if isinstance(result, Exception):
                logging.error(f"Произошла ошибка: {result}")
            else:
                results.extend(result)
    return results

# Определение асинхронной функции для выполнения парсинга всех страниц
async def main(urls, batch_size,parse_func):
    all_listings = await batch_executor(urls, batch_size, parse_func=parse_func)  # Указываем размер партии
    df = pd.DataFrame(all_listings)
    return df



# Выполнение парсинга с использованием aiohttp или Selenium в зависимости от значения параметра method
async def run(urls, method, batch_size):
    try:
        start_time = time.time()

        if method == 'aiohttp':
            df = await main(urls, batch_size, parse_func=parse_with_aiohttp)
        elif method == 'selenium':
            logging.getLogger('selenium.webdriver').setLevel(logging.WARNING)
            df = await main(urls, batch_size, parse_func=parse_with_selenium)

        elapsed_time = time.time() - start_time
        print(f"Execution time: {elapsed_time:.2f} seconds")

        return df
    except Exception as e:
        logger.error(f"Произошла ошибка в функции run: {e}")
        raise

if __name__ == "__main__":
    df = asyncio.run(run(urls, method, batch_size))
    

    
print(f"Время выполнения функции: {time.time() - start_time} секунд")

Execution time: 828.08 seconds
Время выполнения функции: 828.0771050453186 секунд


In [142]:
df

,Название,Цена,Код производителя,Модель,RAM,ROM,Описание
0,Смартфон Xiaomi Redmi Note 13 Pro+ 5G 12/512GB...,43 190,50837,Redmi Note 13 Pro+ 5G 12/512GB,12 Гб,512 Гб,"MediaTek Dimensity 7200-Ultra 2.8 GHz, 6.67"" 2..."
1,Смартфон Apple iPhone 15 128GB Blue MTLG3CH/A,87 444,MTLG3CH/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, 2 Sim, iOS 1..."
2,Смартфон Apple iPhone 15 128GB Yellow MTP23HN/A,87 444,MTP23HN/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, iOS 17, 128 ..."
3,Смартфон Samsung Galaxy XCover 7 6/128GB Black...,37 219,SM-G556BZKDR06,Galaxy XCover 7 6/128GB,6 Гб,128 Гб,"6.6"" 2408x1080, 2 Sim, Android 14, 6 Гб, 128 Г..."
4,Смартфон Samsung Galaxy A34 6/128GB Green SM-A...,20 777,SM-A346ELGACAU,Galaxy A34 6/128GB,6 Гб,128 Гб,"6.6"" 2340x1080, Android 13, 6 Гб, 128 Гб, осно..."
...,...,...,...,...,...,...,...
654,Смартфон Samsung Galaxy S24 8/256GB Yellow SM-...,69 285,None,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A..."
655,Смартфон Samsung Galaxy S24 8/256GB Violet SM-...,69 285,None,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A..."
656,Смартфон Vivo V30 Lite 8/256GB Green,20 286,None,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10..."
657,Смартфон Vivo V30 Lite 8/256GB Black,20 286,None,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10..."


In [143]:
df_kns = df.copy()

In [163]:
# извлечение кода производителя из описания
df_kns['Код производителя из описания'] = df_kns['Описание'].apply(lambda x: str(x).split(',')[-1])

In [169]:
df_kns

,Название,Цена,Код производителя,Модель,RAM,ROM,Описание,Код производителя из описания
0,Смартфон Xiaomi Redmi Note 13 Pro+ 5G 12/512GB...,43 190,50837,Redmi Note 13 Pro+ 5G 12/512GB,12 Гб,512 Гб,"MediaTek Dimensity 7200-Ultra 2.8 GHz, 6.67"" 2...",50837
1,Смартфон Apple iPhone 15 128GB Blue MTLG3CH/A,87 444,MTLG3CH/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, 2 Sim, iOS 1...",MTLG3CH/A
2,Смартфон Apple iPhone 15 128GB Yellow MTP23HN/A,87 444,MTP23HN/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, iOS 17, 128 ...",MTP23HN/A
3,Смартфон Samsung Galaxy XCover 7 6/128GB Black...,37 219,SM-G556BZKDR06,Galaxy XCover 7 6/128GB,6 Гб,128 Гб,"6.6"" 2408x1080, 2 Sim, Android 14, 6 Гб, 128 Г...",SM-G556BZKDR06
4,Смартфон Samsung Galaxy A34 6/128GB Green SM-A...,20 777,SM-A346ELGACAU,Galaxy A34 6/128GB,6 Гб,128 Гб,"6.6"" 2340x1080, Android 13, 6 Гб, 128 Гб, осно...",SM-A346ELGACAU
...,...,...,...,...,...,...,...,...
654,Смартфон Samsung Galaxy S24 8/256GB Yellow SM-...,69 285,None,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A...",SM-S921BZYGSKZ
655,Смартфон Samsung Galaxy S24 8/256GB Violet SM-...,69 285,None,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A...",SM-S921BZVGSKZ
656,Смартфон Vivo V30 Lite 8/256GB Green,20 286,None,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10...",5666661
657,Смартфон Vivo V30 Lite 8/256GB Black,20 286,None,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10...",5666539


In [179]:
# подстановка кодов производителя из описания для тех случаев, где коды производителя не были указаны в списке характеристик на сайте
df_kns['Код производителя'] = df_kns['Код производителя'].fillna(df_kns['Код производителя из описания']) 

In [189]:
# удаление печатных символов из цены и перевод к типу int
df_kns['Цена'] = df_kns['Цена'].apply(lambda x: int(x.replace('\xa0', '')))

In [205]:
# извлечение названия цвета из описания
df_kns['Цвет'] = df_kns['Описание'].apply(lambda x: str(x).split(',')[-3] if isinstance(x, str) and len(str(x).split(',')) >= 3 else None)

In [241]:
df_kns['Цвет'].unique()

array([' черный', ' голубой', ' желтый', ' зеленый', ' темная ночь',
       ' фиолетовый', ' темно-синий', ' серебристый', ' синий', ' серый',
       ' золотой', ' белый', ' черный титан', ' черный (темная ночь)',
       ' лаванда', ' светло-голубой', ' бежевый титан',
       ' белый (жемчужина рококо)', ' черный фантом', ' графит',
       ' розовый', ' синий титан', None, ' белый титан', ' красный',
       ' кремовый', ' бежевый', ' мятный', ' черный - зеленый',
       ' сияющая звезда', ' белый фантом', ' коричневый',
       ' белый (сияющая звезда)', ' титан', ' бирюзовый', ' лунная тень',
       ' альпийский зеленый', ' сиреневый', ' оранжевый', ' медный',
       ' голубой матовый/медный'], dtype=object)

In [243]:
df_kns[df_kns['Цвет'].str.contains('\d', na=False)]['Описание'].to_list()

[]

In [246]:
df_kns

,Название,Цена,Код производителя,Модель,RAM,ROM,Описание,Код производителя из описания,Цвет
0,Смартфон Xiaomi Redmi Note 13 Pro+ 5G 12/512GB...,43190,50837,Redmi Note 13 Pro+ 5G 12/512GB,12 Гб,512 Гб,"MediaTek Dimensity 7200-Ultra 2.8 GHz, 6.67"" 2...",50837,черный
1,Смартфон Apple iPhone 15 128GB Blue MTLG3CH/A,87444,MTLG3CH/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, 2 Sim, iOS 1...",MTLG3CH/A,голубой
2,Смартфон Apple iPhone 15 128GB Yellow MTP23HN/A,87444,MTP23HN/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, iOS 17, 128 ...",MTP23HN/A,желтый
3,Смартфон Samsung Galaxy XCover 7 6/128GB Black...,37219,SM-G556BZKDR06,Galaxy XCover 7 6/128GB,6 Гб,128 Гб,"6.6"" 2408x1080, 2 Sim, Android 14, 6 Гб, 128 Г...",SM-G556BZKDR06,черный
4,Смартфон Samsung Galaxy A34 6/128GB Green SM-A...,20777,SM-A346ELGACAU,Galaxy A34 6/128GB,6 Гб,128 Гб,"6.6"" 2340x1080, Android 13, 6 Гб, 128 Гб, осно...",SM-A346ELGACAU,зеленый
...,...,...,...,...,...,...,...,...,...
654,Смартфон Samsung Galaxy S24 8/256GB Yellow SM-...,69285,SM-S921BZYGSKZ,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A...",SM-S921BZYGSKZ,желтый
655,Смартфон Samsung Galaxy S24 8/256GB Violet SM-...,69285,SM-S921BZVGSKZ,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A...",SM-S921BZVGSKZ,фиолетовый
656,Смартфон Vivo V30 Lite 8/256GB Green,20286,5666661,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10...",5666661,зеленый
657,Смартфон Vivo V30 Lite 8/256GB Black,20286,5666539,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10...",5666539,черный


In [231]:
mask = df_kns['RAM'].str.contains('\d', na=False)
# заменяем значения в столбце 'Цвет' на основе значений из столбца 'Описание'
df_kns.loc[mask, 'RAM'] = df_kns.loc[mask, 'Описание'].apply(lambda x: str(x).split(',')[-2] if isinstance(x, str) and len(str(x).split(',')) >= 3 else None)

In [250]:
df_kns['ROM'].unique()

array(['512 Гб', '128 Гб', '1024 Гб', '256 Гб', '64 Гб', '32 Гб', None],
      dtype=object)

In [314]:
df_kns[~df_kns['Название'].str.contains('GB|TB', case=False)]

,Название,Цена,Код производителя,Модель,RAM,ROM,Описание,Код производителя из описания,Цвет,RAM/ROM


In [304]:
# извлечение из названия параметров RAM/ROM
df_kns['RAM/ROM'] = df_kns['Название'].str.extract(r'(\d+/?\d*\s*(?:Tb|Gb))', flags=re.IGNORECASE,expand=False)

In [308]:
df_kns

,Название,Цена,Код производителя,Модель,RAM,ROM,Описание,Код производителя из описания,Цвет,RAM/ROM
0,Смартфон Xiaomi Redmi Note 13 Pro+ 5G 12/512GB...,43190,50837,Redmi Note 13 Pro+ 5G 12/512GB,12 Гб,512 Гб,"MediaTek Dimensity 7200-Ultra 2.8 GHz, 6.67"" 2...",50837,черный,12/512GB
1,Смартфон Apple iPhone 15 128GB Blue MTLG3CH/A,87444,MTLG3CH/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, 2 Sim, iOS 1...",MTLG3CH/A,голубой,128GB
2,Смартфон Apple iPhone 15 128GB Yellow MTP23HN/A,87444,MTP23HN/A,iPhone 15 128GB,6 Гб,128 Гб,"Apple A16 Bionic, 6.1"" 2556x1179, iOS 17, 128 ...",MTP23HN/A,желтый,128GB
3,Смартфон Samsung Galaxy XCover 7 6/128GB Black...,37219,SM-G556BZKDR06,Galaxy XCover 7 6/128GB,6 Гб,128 Гб,"6.6"" 2408x1080, 2 Sim, Android 14, 6 Гб, 128 Г...",SM-G556BZKDR06,черный,6/128GB
4,Смартфон Samsung Galaxy A34 6/128GB Green SM-A...,20777,SM-A346ELGACAU,Galaxy A34 6/128GB,6 Гб,128 Гб,"6.6"" 2340x1080, Android 13, 6 Гб, 128 Гб, осно...",SM-A346ELGACAU,зеленый,6/128GB
...,...,...,...,...,...,...,...,...,...,...
654,Смартфон Samsung Galaxy S24 8/256GB Yellow SM-...,69285,SM-S921BZYGSKZ,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A...",SM-S921BZYGSKZ,желтый,8/256GB
655,Смартфон Samsung Galaxy S24 8/256GB Violet SM-...,69285,SM-S921BZVGSKZ,None,None,None,"Samsung Exynos 2400 3.2 GHz, 6.2"" 2340x1080, A...",SM-S921BZVGSKZ,фиолетовый,8/256GB
656,Смартфон Vivo V30 Lite 8/256GB Green,20286,5666661,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10...",5666661,зеленый,8/256GB
657,Смартфон Vivo V30 Lite 8/256GB Black,20286,5666539,None,None,None,"Qualcomm Snapdragon 685 2.8 GHz, 6.67"" 2400x10...",5666539,черный,8/256GB


In [310]:
df_kns.to_excel('kns телефоны.xlsx', index=False)

In [607]:
# Метод aiohttp или selenium
method = 'selenium'

In [343]:
# ссылки на страницы поиска телефонов сайта Regard
urls2 = [f'https://www.regard.ru/catalog/1252/mobilnye-telefony?page={i}&q=eyJzb3J0IjpbIm9yZGVyQnlQcmljZSIsImFzYyJdfQ' for i in range(1,39+1)]


In [351]:
urls = urls2
batch_size = 1

In [353]:
start_time = time.time()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
nest_asyncio.apply()



@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(aiohttp.ClientError)
)
async def fetch(session, url, ):
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'ru-RU,ru;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1'
    }
    proxy = "http://127.0.0.1:50445"
    async with session.get(url, headers=headers, proxy=proxy) as response:
        response.raise_for_status()  # Это вызовет исключение для ненулевых статусов
        return await response.text()


@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(Exception))

async def fetch_with_selenium(url, proxy="127.0.0.1:50445"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        
    driver_path = 'C:\\Program Files\\Google\\Chrome\\Application\\chromedriver-win64\\chromedriver.exe'
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        await asyncio.sleep(random.uniform(6, 12))  # Рандомная задержка
        html_content = driver.page_source
    finally:
        driver.quit()
    return html_content


async def parse_with_aiohttp(url):
    async with aiohttp.ClientSession() as session:
        html_content = await fetch(session, url)
        await asyncio.sleep(random.uniform(4, 7))
        
        soup = BeautifulSoup(html_content, 'lxml')
        items = soup.select('div.CardText_wrap__YYHuc.CardText_listing__6mqXC a.CardText_link__C_fPZ.link_black')
 


        listings = []
        for idx, item in enumerate(items):
            title = item.get_text(strip=True)
            link = item['href']

            # Проверяем, что элементы rating_oblast1, text1 и gorod1 существуют и добавляем их в listings
            #price = prices[idx].get_text(strip=True) if idx < len(price) else None


            listings.append({'Название': title,'Ссылка':link})
        return listings
    

async def parse_with_selenium(url):
    html_content = await fetch_with_selenium(url)
    soup = BeautifulSoup(html_content, 'lxml')
    
    items = soup.select('div.CardText_wrap__YYHuc.CardText_listing__6mqXC a.CardText_link__C_fPZ.link_black')
 
    listings = []
    for idx, item in enumerate(items):
        title = item.get_text(strip=True)
        link = item['href']

        # Проверяем, что элементы rating_oblast1, text1 и gorod1 существуют и добавляем их в listings
        #price = prices[idx].get_text(strip=True) if idx < len(price) else None


        listings.append({'Название': title,'Ссылка':link})
    return listings



async def batch_executor(urls, batch_size=2, parse_func=None):
    results = []
    for i in range(0, len(urls), batch_size):
        batch = urls[i:i + batch_size]
        tasks = [parse_func(url) for url in batch]
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        for result in batch_results:
            if isinstance(result, Exception):
                logging.error(f"Произошла ошибка: {result}")
            else:
                results.extend(result)
    return results

async def main(urls, batch_size,parse_func):
    all_listings = await batch_executor(urls, batch_size, parse_func=parse_func)  # Указываем размер партии
    df = pd.DataFrame(all_listings)
    return df





async def run(urls, method, batch_size):
    try:
        start_time = time.time()

        if method == 'aiohttp':
            df = await main(urls, batch_size, parse_func=parse_with_aiohttp)
        elif method == 'selenium':
            logging.getLogger('selenium.webdriver').setLevel(logging.WARNING)
            df = await main(urls, batch_size, parse_func=parse_with_selenium)

        elapsed_time = time.time() - start_time
        print(f"Execution time: {elapsed_time:.2f} seconds")

        return df
    except Exception as e:
        logger.error(f"Произошла ошибка в функции run: {e}")
        raise

if __name__ == "__main__":
    df = asyncio.run(run(urls, method, batch_size))
    

    
    
    
    
    
print(f"Время выполнения функции: {time.time() - start_time} секунд")

ERROR:root:An error occurred: RetryError[<Future at 0x1f0afc58e50 state=finished raised ClientResponseError>]
ERROR:root:An error occurred: RetryError[<Future at 0x1f0ae560970 state=finished raised ClientResponseError>]
ERROR:root:An error occurred: RetryError[<Future at 0x1f0ad2ec9d0 state=finished raised ClientResponseError>]
ERROR:root:An error occurred: RetryError[<Future at 0x1f0ae517eb0 state=finished raised ClientResponseError>]


Execution time: 305.43 seconds
Время выполнения функции: 305.43343114852905 секунд


In [354]:
df_regard_links = df.copy()

In [357]:
df_regard_links

,Название,Ссылка
0,Телефон INOI 101 Black,/product/70987/telefon-inoi-101-black
1,Телефон Digma Linx A106 Black,/product/67681/telefon-digma-linx-a106-black
2,Телефон Fplus F170L Light Blue,/product/475457/telefon-fplus-f170l-light-blue
3,Телефон INOI 105 Black,/product/73579/telefon-inoi-105-black
4,Телефон INOI 108R Black,/product/70983/telefon-inoi-108r-black
...,...,...
827,Смартфон Apple iPhone 14 Pro Max 1Tb Silver (M...,/product/470530/smartfon-apple-iphone-14-pro-m...
828,Смартфон Apple iPhone 14 Pro Max 1Tb Space Bla...,/product/473072/smartfon-apple-iphone-14-pro-m...
829,Смартфон Apple iPhone 15 Pro Max 1Tb Black Tit...,/product/78279/smartfon-apple-iphone-15-pro-ma...
830,Смартфон Apple iPhone 15 Pro Max 1Tb Natural T...,/product/655931/smartfon-apple-iphone-15-pro-m...


In [359]:
df_regard_links['regard'] = 'https://www.regard.ru' + df_regard_links['Ссылка']

In [365]:
df_regard_links = df_regard_links.rename(columns={'regard':'Ссылка на телефон'})

In [768]:
df_regard_links.to_excel('regard ссылки на телефоны.xlsx', index=False)

In [10]:
df_regard_links = pd.read_excel('regard ссылки на телефоны.xlsx')

In [12]:
df_regard_links

,Название,Ссылка,Ссылка на телефон
0,Телефон INOI 101 Black,/product/70987/telefon-inoi-101-black,https://www.regard.ru/product/70987/telefon-in...
1,Телефон Digma Linx A106 Black,/product/67681/telefon-digma-linx-a106-black,https://www.regard.ru/product/67681/telefon-di...
2,Телефон Fplus F170L Light Blue,/product/475457/telefon-fplus-f170l-light-blue,https://www.regard.ru/product/475457/telefon-f...
3,Телефон INOI 105 Black,/product/73579/telefon-inoi-105-black,https://www.regard.ru/product/73579/telefon-in...
4,Телефон INOI 108R Black,/product/70983/telefon-inoi-108r-black,https://www.regard.ru/product/70983/telefon-in...
...,...,...,...
827,Смартфон Apple iPhone 14 Pro Max 1Tb Silver (M...,/product/470530/smartfon-apple-iphone-14-pro-m...,https://www.regard.ru/product/470530/smartfon-...
828,Смартфон Apple iPhone 14 Pro Max 1Tb Space Bla...,/product/473072/smartfon-apple-iphone-14-pro-m...,https://www.regard.ru/product/473072/smartfon-...
829,Смартфон Apple iPhone 15 Pro Max 1Tb Black Tit...,/product/78279/smartfon-apple-iphone-15-pro-ma...,https://www.regard.ru/product/78279/smartfon-a...
830,Смартфон Apple iPhone 15 Pro Max 1Tb Natural T...,/product/655931/smartfon-apple-iphone-15-pro-m...,https://www.regard.ru/product/655931/smartfon-...


In [31]:
# Метод aiohttp или selenium
method = 'aiohttp'
urls = df_regard_links['Ссылка на телефон'].to_list()
batch_size = 1

In [33]:
start_time = time.time()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
nest_asyncio.apply()


@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=2, max=6),
    retry=tenacity.retry_if_exception_type(aiohttp.ClientError)
)
async def fetch(session, url):
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'ru-RU,ru;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1'
    }
    proxy = "http://127.0.0.1:50445"
    async with session.get(url, headers=headers, proxy=proxy) as response:
        response.raise_for_status()  # Это вызовет исключение для ненулевых статусов
        return await response.text()


@tenacity.retry(
    stop=tenacity.stop_after_attempt(3),
    wait=tenacity.wait_exponential(multiplier=1, min=4, max=8),
    retry=tenacity.retry_if_exception_type(Exception))

async def fetch_with_selenium(url, proxy="127.0.0.1:50445"):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    if proxy:
        chrome_options.add_argument(f'--proxy-server={proxy}')
        
    driver_path = 'C:\\Program Files\\Google\\Chrome\\Application\\chromedriver-win64\\chromedriver.exe'
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)
        await asyncio.sleep(random.uniform(7, 15))  # Рандомная задержка
        html_content = driver.page_source
    finally:
        driver.quit()
    return html_content


async def parse_with_aiohttp(url):
    async with aiohttp.ClientSession() as session:
        html_content = await fetch(session, url)
        await asyncio.sleep(random.uniform(6, 12))
        
        soup = BeautifulSoup(html_content, 'lxml')
        listings = []
        def get_text_or_none(selector):
            element = soup.select_one(selector)
            return element.get_text(strip=True) if element else None
        def search_inf_by_text(text, tag='a',class_='CharacteristicsItem_valueLink__0u9tP'):
            items = soup.find_all('div', class_='CharacteristicsItem_item__QnlK2')
            for item in items:
                name_div = item.find('div', class_='CharacteristicsItem_name__Q7B8V')
                if name_div:
                    name = name_div.text
 
                    if text in name:

                        value_span = item.find(tag, class_=class_)
                        if value_span:
                            value = value_span.text.strip()
                        else:
                            value = np.nan
                        return value
        price = get_text_or_none('span.PriceBlock_price__j_PbO')
        main_name = get_text_or_none('h1.Product_title__42hYI')
        PN = search_inf_by_text(text='Код производителя', tag='span', class_='CharacteristicsItem_valueData__d_mHr')
        model = search_inf_by_text(text='Модель', class_='CharacteristicsItem_valueLink__0u9tP')
        color = search_inf_by_text(text='Цвет', tag='span',class_='CharacteristicsItem_valueData__d_mHr')
        RAM = search_inf_by_text(text='Объём оперативной памяти', class_='CharacteristicsItem_valueLink__0u9tP')
        ROM = search_inf_by_text(text='Объём встроенной памяти', class_='CharacteristicsItem_valueLink__0u9tP')
    
        listings.append({'Ссылка':url, 'Название': main_name,'Цена':price, 'Код производителя':PN, 'Модель':model,"RAM":RAM, 'ROM':ROM, 'Цвет':color})
        return listings
    

async def parse_with_selenium(url):
    html_content = await fetch_with_selenium(url)
    soup = BeautifulSoup(html_content, 'lxml')
    listings = []
    def get_text_or_none(selector):
        element = soup.select_one(selector)
        return element.get_text(strip=True) if element else None
    
    price = get_text_or_none('span.PriceBlock_price__j_PbO')
    main_name = get_text_or_none('h1.Product_title__42hYI')
    PN = search_inf_by_text(text='Код производителя', tag='span', class_='CharacteristicsItem_valueData__d_mHr')
    model = search_inf_by_text(text='Модель', class_='CharacteristicsItem_valueLink__0u9tP')
    color = search_inf_by_text(text='Цвет', tag='span',class_='CharacteristicsItem_valueData__d_mHr')
    RAM = search_inf_by_text(text='Объём оперативной памяти', class_='CharacteristicsItem_valueLink__0u9tP')
    ROM = search_inf_by_text(text='Объём встроенной памяти', class_='CharacteristicsItem_valueLink__0u9tP')
    
    listings.append({'Ссылка':url, 'Название': main_name,'Цена':price, 'Код производителя':PN, 'Модель':model,"RAM":RAM, 'ROM':ROM, 'Цвет':color})
    return listings


async def batch_executor(urls, batch_size=2, parse_func=None):
    results = []
    for i in range(0, len(urls), batch_size):
        batch = urls[i:i + batch_size]
        tasks = [parse_func(url) for url in batch]
        batch_results = await asyncio.gather(*tasks, return_exceptions=True)
        for result in batch_results:
            if isinstance(result, Exception):
                logging.error(f"Произошла ошибка: {result}")
            else:
                results.extend(result)
    return results

async def main(urls, batch_size,parse_func):
    all_listings = await batch_executor(urls, batch_size, parse_func=parse_func)  # Указываем размер партии
    df = pd.DataFrame(all_listings)
    return df



async def run(urls, method, batch_size):
    try:
        start_time = time.time()

        if method == 'aiohttp':
            df = await main(urls, batch_size, parse_func=parse_with_aiohttp)
        elif method == 'selenium':
            logging.getLogger('selenium.webdriver').setLevel(logging.WARNING)
            df = await main(urls, batch_size, parse_func=parse_with_selenium)

        elapsed_time = time.time() - start_time
        print(f"Execution time: {elapsed_time:.2f} seconds")

        return df
    except Exception as e:
        logger.error(f"Произошла ошибка в функции run: {e}")
        raise

if __name__ == "__main__":
    df = asyncio.run(run(urls, method, batch_size))
    

    
    
    
    
    
print(f"Время выполнения функции: {time.time() - start_time} секунд")

ERROR:root:An error occurred: RetryError[<Future at 0x22b420d98e0 state=finished raised ClientResponseError>]
ERROR:root:An error occurred: RetryError[<Future at 0x22b41ab8a60 state=finished raised ClientResponseError>]
ERROR:root:An error occurred: RetryError[<Future at 0x22b42ad12e0 state=finished raised ClientResponseError>]
ERROR:root:An error occurred: RetryError[<Future at 0x22b42aec910 state=finished raised ClientResponseError>]
ERROR:root:An error occurred: RetryError[<Future at 0x22b436fd490 state=finished raised ClientResponseError>]


Execution time: 9900.81 seconds
Время выполнения функции: 9900.807436704636 секунд


In [34]:
df

,Ссылка,Название,Цена,Код производителя,Модель,RAM,ROM,Цвет
0,https://www.regard.ru/product/70987/telefon-in...,Телефон INOI 101 Black,880₽,None,None,None,None,чёрный
1,https://www.regard.ru/product/67681/telefon-di...,Телефон Digma Linx A106 Black,890₽,LT1065PMBK,None,32 Мб,32 Мб,серый
2,https://www.regard.ru/product/475457/telefon-f...,Телефон Fplus F170L Light Blue,900₽,None,None,None,None,голубой
3,https://www.regard.ru/product/73579/telefon-in...,Телефон INOI 105 Black,1000₽,None,None,None,None,чёрный
4,https://www.regard.ru/product/70983/telefon-in...,Телефон INOI 108R Black,1060₽,None,None,None,None,чёрный
...,...,...,...,...,...,...,...,...
822,https://www.regard.ru/product/470530/smartfon-...,Смартфон Apple iPhone 14 Pro Max 1Tb Silver (M...,194490₽,MQ9L3J/A,None,6 Гб,1 Тб,серебристый
823,https://www.regard.ru/product/473072/smartfon-...,Смартфон Apple iPhone 14 Pro Max 1Tb Space Bla...,194490₽,MQ9K3J/A,None,6 Гб,1 Тб,чёрный
824,https://www.regard.ru/product/78279/smartfon-a...,Смартфон Apple iPhone 15 Pro Max 1Tb Black Tit...,204240₽,MU2X3ZA/A,Apple iPhone 15 Pro Max,None,1 Тб,чёрный
825,https://www.regard.ru/product/655931/smartfon-...,Смартфон Apple iPhone 15 Pro Max 1Tb Natural T...,212120₽,MU713J/A,Apple iPhone 15 Pro Max,None,1 Тб,серый


In [42]:
len(urls)

832

In [35]:
df_regard = df.copy()

In [73]:
df_regard[df_regard['Цена'].isna()]

,Ссылка,Название,Цена,Код производителя,Модель,RAM,ROM,Цвет


In [89]:
df_regard['Цена_число'] = df_regard['Цена'].str.extract(r'(\d+)')

In [100]:
df_regard = df_regard[df_regard['Цена_число'].notna()]
df_regard['Цена_число'] = df_regard['Цена_число'].astype(int)

C:\Users\perfe\AppData\Local\Temp\ipykernel_16800\2155121778.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regard['Цена_число'] = df_regard['Цена_число'].astype(int)


In [120]:
df_regard = df_regard.drop('Цена', axis=1).rename(columns={'Цена_число':'Цена'})

In [122]:
df_regard

,Ссылка,Название,Код производителя,Модель,RAM,ROM,Цвет,Цена
0,https://www.regard.ru/product/70987/telefon-in...,Телефон INOI 101 Black,None,None,None,None,чёрный,880
1,https://www.regard.ru/product/67681/telefon-di...,Телефон Digma Linx A106 Black,LT1065PMBK,None,32 Мб,32 Мб,серый,890
2,https://www.regard.ru/product/475457/telefon-f...,Телефон Fplus F170L Light Blue,None,None,None,None,голубой,900
3,https://www.regard.ru/product/73579/telefon-in...,Телефон INOI 105 Black,None,None,None,None,чёрный,1000
4,https://www.regard.ru/product/70983/telefon-in...,Телефон INOI 108R Black,None,None,None,None,чёрный,1060
...,...,...,...,...,...,...,...,...
822,https://www.regard.ru/product/470530/smartfon-...,Смартфон Apple iPhone 14 Pro Max 1Tb Silver (M...,MQ9L3J/A,None,6 Гб,1 Тб,серебристый,194490
823,https://www.regard.ru/product/473072/smartfon-...,Смартфон Apple iPhone 14 Pro Max 1Tb Space Bla...,MQ9K3J/A,None,6 Гб,1 Тб,чёрный,194490
824,https://www.regard.ru/product/78279/smartfon-a...,Смартфон Apple iPhone 15 Pro Max 1Tb Black Tit...,MU2X3ZA/A,Apple iPhone 15 Pro Max,None,1 Тб,чёрный,204240
825,https://www.regard.ru/product/655931/smartfon-...,Смартфон Apple iPhone 15 Pro Max 1Tb Natural T...,MU713J/A,Apple iPhone 15 Pro Max,None,1 Тб,серый,212120


In [124]:
df_regard.to_excel('regard телефоны.xlsx', index=False)